In [2]:
# jupyter notebook 환경에서 spark를 사용하기 위한 모듈
## Windows
# import findspark
# findspark.init('C:/spark/spark-3.0.1-bin-hadoop2.7')

In [1]:
# pyspark import
import pyspark
from pyspark import SparkConf,SparkContext
from pyspark.sql.context import SQLContext
from pyspark.sql import SparkSession

In [2]:
# pyspark 연결하기
# 꼬이면 kernel restart
conf= pyspark.SparkConf().setAppName('appName').setMaster('local')
sc= pyspark.SparkContext(conf= conf)
spark= SparkSession(sc)

22/04/23 00:38:12 WARN Utils: Your hostname, baesuhan-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.35.253 instead (on interface en0)
22/04/23 00:38:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/23 00:38:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# JSON 데이터 생성하기

In [7]:
stringJSONRDD = sc.parallelize((
"""{"id": "123", "name": "Katie", "age": 19, "eyeColor": "brown"}""",
    """{"id": "234", "name": "Michael", "age": 22, "eyeColor": "green"}""",
    """{"id": "345", "name": "Simone", "age": 23, "eyeColor": "blue"}"""
))

In [8]:
# 데이터프레임 생성하기
swimmersJSON= spark.read.json(stringJSONRDD)

In [9]:
# 임시 테이블 생성하기
swimmersJSON.createOrReplaceTempView("swimmersJSON")

In [10]:
swimmersJSON.show() # dataframe을 콘솔에 출력하는 함수

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



# SQL 쿼리

In [11]:
spark.sql("select * from swimmersJSON").collect()
# collect함수는 모든 행을 리턴하므로 작은 데이터프레임에 대해 사용하는 것이 더 좋다.
# n개의 행을 리턴할 때에는 일반적으로 take(n)이나 show(n)을 사용하는 것이 더 좋다.

[Row(age=19, eyeColor='brown', id='123', name='Katie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

In [12]:
# 스키마 정의 확인하기
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



In [13]:
# 위 schema에서 명시된 type을 명시하고 싶을 때 사용하는 방법
from pyspark.sql.types import *

stringCSVRDD = sc.parallelize([
    (123, 'Katie', 19, 'brown'),
    (234, 'Michael', 22, 'green'),
    (345, 'Simone', 23, 'blue')
])

# 스키마 명시하기
schema = StructType([
    StructField("id", LongType(), True),
    StructField("name", StringType(), True),
    StructField("age", LongType(), True),
    StructField("eyeColor", StringType(), True)
])

In [14]:
# RDD에 스키마를 적용하고 데이터프레임을 생성하기
swimmers = spark.createDataFrame(stringCSVRDD, schema)

# 데이터프레임을 이용해 임시 뷰를 생성
swimmers.createOrReplaceTempView("swimmers")

In [15]:
swimmers.printSchema() # schema에서 명시된 순서와 타입대로 정렬되었음을 알 수 있다.

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



# 데이터프레임 API로 쿼리하기

In [16]:
swimmers.count() # 행 개수 세기

3

In [17]:
# age가 22인 데이터의 id와 age를 출력하기
swimmers.select("id", "age").filter("age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [18]:
# 다른 방식
swimmers.select(swimmers.id, swimmers.age).filter(swimmers.age == 22).show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [19]:
# eyeColor가 b로 시작하는 데이터의 name, eyeColor 칼럼을 얻는다.
swimmers.select("name", "eyeColor").filter("eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



# SQL 쿼리

In [20]:
spark.sql("select count(1) from swimmers").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



In [21]:
spark.sql("select id, age from swimmers where age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [22]:
spark.sql("select name, eyeColor from swimmers where eyeColor like 'b%'").show() # 눈 색깔이 b로 시작하는 사람 찾기

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



In [23]:
# 출발지 여행 데이터셋
flightPerFilePath = "./dataset/departuredelays.csv"
airportsFilePath = "./dataset/airport-codes-na.txt"

In [24]:
airports = spark.read.csv(airportsFilePath, header = 'true', inferSchema='true', sep='\t')
airports.createOrReplaceTempView("airports")

AnalysisException: Path does not exist: file:/Users/soohan/pyspark/dataset/airport-codes-na.txt

In [25]:
flightPerf = spark.read.csv(flightPerFilePath, header = 'true')
flightPerf.createOrReplaceTempView("FlightPerformance")

AnalysisException: Path does not exist: file:/Users/soohan/pyspark/dataset/departuredelays.csv

In [8]:
flightPerf.cache()

DataFrame[date: string, delay: string, distance: string, origin: string, destination: string]

In [26]:
# 비행 성능 데이터셋 및 공항 데이터셋 조인하기
spark.sql("""
    select a.City,
    f.origin,
    sum(f.delay) as Delays
    from FlightPerformance f
    join airports a
    on a.IATA = f.origin
    where a.State = 'WA'
    group by a.City, f.origin
    order by sum(f.delay) desc"""
         ).show()

AnalysisException: Table or view not found: FlightPerformance; line 5 pos 9;
'Sort ['sum('f.delay) DESC NULLS LAST], true
+- 'Aggregate ['a.City, 'f.origin], ['a.City, 'f.origin, 'sum('f.delay) AS Delays#148]
   +- 'Filter ('a.State = WA)
      +- 'Join Inner, ('a.IATA = 'f.origin)
         :- 'SubqueryAlias f
         :  +- 'UnresolvedRelation [FlightPerformance], [], false
         +- 'SubqueryAlias a
            +- 'UnresolvedRelation [airports], [], false
